In [1]:
import pandas as pd
import os

path = './datasets/'

if os.path.exists(path):
    # List all files in the folder and filter for CSV files
    csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]
    
    # Load each CSV file into a dictionary of dataframes
    dataframes = {file: pd.read_csv(os.path.join(path, file)) for file in csv_files}

    # Display names of the loaded files and their contents
    for file_name, df in dataframes.items():
        print(f"Data from {file_name}:")
        display(df.head())  # Show the first 5 rows of each dataframe
else:
    print("The datasets folder does not exist. Please check the path.")

Data from data_set_1.csv:


,1,0.7197905348441712,0.3172602515682258
0,2,0.718572,0.308538
1,3,0.717821,0.303025
2,4,0.725701,0.363205
3,5,0.724039,0.350726
4,6,0.722917,0.342374


Data from data_set_1_method1_sel_1.csv:


,50060,0.9103867662653103,0.4380915653448884
0,50707,0.911345,0.439669
1,53020,0.908710,0.430283
2,53059,0.904187,0.430096
3,459321,0.913391,0.421065
4,51662,0.906532,0.432540


Data from data_set_1_method1_sel_2.csv:


...

,472353,1.0088372414609881,0.7839094601569989
0,451678,1.008220,0.784495
1,472016,1.008935,0.784241
2,472355,1.009057,0.784652
3,452876,1.008140,0.784199
4,451979,1.008315,0.784784


Data from data_set_2_method2_sel_1.csv:


,318871,1.0165072634549983,0.8175560435673671
0,361070,1.014309,0.813463
1,304871,1.017619,0.822101
2,90743,0.886789,0.563786
3,298435,1.018279,0.825885
4,379247,1.013274,0.809364


Data from data_set_2_method2_sel_2.csv:


,405088,0.3876397123321345,0.0560581165699712
0,405051,0.387918,0.057605
1,384708,0.389587,0.065535
2,283153,0.390088,0.067946
3,133830,0.391395,0.073919
4,171979,0.391812,0.076006
